In [7]:
# @author Jorge Luiz, apoio moral Renan Balbinot
# UTFPR 2017


# imports
import cv2
import numpy as np
import glob

# globais
glob_printstep = False    # mostra os passos de processamento foram executados para cada imagem
glob_printresult = True  # mostra os resultados para cada imagem

# variáveis
input_dir = ""
output_dir = "out/"
avg_error = 0
max_error = 0


# funções helper para prints
def printstep(string):
    if(glob_printstep):
        print(string)
        
def printresult(string):
    if(glob_printresult):
        print(string)


# lista de nomes de arquivos a avaliar
img_list = glob.glob("*.jpg")
print(img_list)
print()

# loop que trabalha a lista de imagens
for img_path in img_list:

    # abre a imagem e extrai o tamanho dela, extensão e nome do arquivo
    img = cv2.imread(img_path)
    img_name = img_path.split(".")[0]
    img_h, img_w, img_channels = img.shape  # as imagens podem ter diferentes resoluções
    
    # define alguns caminhos importantes - melhor legibilidade no código
    crop_path = output_dir + img_name + "_1_crop.png"
    output_imgpath = output_dir + img_name + "_"
    
    printstep("processando %s..." % img_name)
    
    # extrai um quadrado de 68x68 no canto inferior direito - funciona para todas as resoluções
    crop_img = img[img_h-91:img_h-91+68, img_w-83:img_w-83+68]
    cv2.imwrite(crop_path, crop_img)
    printstep("executado passo 1: crop")


    # converte para outro espaço de cores
    colorspace_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2Lab)
    colorspace_img = colorspace_img[:,:,2]  # extrai o canal
    cv2.imwrite(output_imgpath + "2_colorspace.png", colorspace_img)
    printstep("executado passo 2: conversão para outro espaço de cor")
    
   
    # binariza a imagem usando otsu
    threshold_value, binary_img = cv2.threshold(colorspace_img, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imwrite(output_imgpath + "3_binary.png", binary_img)
    printstep("executado passo 3: binarização")
    
    
    # aplica máscara
    mask_img = cv2.imread('~mask.png', 0)
    mask_img = cv2.bitwise_and(binary_img, binary_img, mask=mask_img)
    cv2.imwrite(output_imgpath + "4_mask.png", mask_img)
    printstep("executado passo 4: aplicada máscara")
    
    
    # aplica morfologia
    kernel = np.ones((3,3), np.uint8)
    morph_img = cv2.morphologyEx(mask_img, cv2.MORPH_OPEN, kernel, iterations=2)
    cv2.imwrite(output_imgpath + "5_morph.png", morph_img)
    printstep("executado passo 5: aplicada abertura")
    
    
    # detecta contorno
    contorno_img, contours, hierarchy = cv2.findContours(morph_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    # desenha o contorno, apenas para fins de visualização
    contorno_img = cv2.imread(crop_path)
    cv2.drawContours(contorno_img, contours, -1, (0,255,0), 1)
    cv2.imwrite(output_imgpath + "6_contorno.png", contorno_img)
    printstep("executado passo 5: detectado contorno")
    
    
    # desenha a elipse
    contours = contours[0]
    ellipse = cv2.fitEllipse(contours)
    # desenha a elipse, apenas para fins de visualização
    cv2.ellipse(crop_img,ellipse,(0,255,0),1)
    cv2.imwrite(output_imgpath + "7_elipse.png", crop_img)
    printstep("executado passo 7: traçada elipse")
    
    
    # análise dos resultados
    printresult("calculando %s..." % img_name)
    ang_obtido = ellipse[2]
    if(ang_obtido > 90):
        ang_obtido = 180 - ang_obtido
    
    # ang_esperado apenas deve ser usado para as imagens de debugging
    ang_esperado = int(img_name.split("_")[1])
    ang_esperado /= 10
    error = abs(ang_esperado - ang_obtido)
    printresult("ang esp: %.1f" % ang_esperado)
    printresult("ang obt: %.1f" % ang_obtido)
    
    # mais lógica de debugging
    printresult("erro: %.1f" % error)
    avg_error += error
    if(error > max_error):
        max_error = error
    
    if(glob_printstep or glob_printresult): # imprime uma newline entre cada iteração do loop se estiver debugando
        print()
    

# saída do loop. print dos resultados finais
print("Analisadas %d imagens..." % len(img_list))
print("Erro médio: %.1f" % float(avg_error/len(img_list)))
print("Maior erro: %.1f" % max_error)

['BL_101.jpg', 'BL_697.jpg', 'BMx_464.jpg', 'BMy_464.jpg', 'BM_039.jpg', 'BM_079.jpg', 'BM_266.jpg', 'BM_590.jpg', 'DI_222.jpg', 'DI_393.jpg', 'EVx_266.jpg', 'EV_266.jpg', 'EV_492.jpg', 'IC_799.jpg', 'PW_507.jpg', 'PW_697.jpg', 'SS_020.jpg', 'SS_079.jpg', 'SS_178.jpg', 'SS_248.jpg', 'SS_266.jpg', 'SS_408.jpg', 'SS_436.jpg', 'SS_880.jpg', 'SWx_617.jpg', 'SW_617.jpg', 'WW_178.jpg', 'WW_203.jpg']

calculando BL_101...
ang esp: 10.1
ang obt: 10.7
erro: 0.6

calculando BL_697...
ang esp: 69.7
ang obt: 69.8
erro: 0.1

calculando BMx_464...
ang esp: 46.4
ang obt: 46.3
erro: 0.1

calculando BMy_464...
ang esp: 46.4
ang obt: 45.6
erro: 0.8

calculando BM_039...
ang esp: 3.9
ang obt: 3.4
erro: 0.5

calculando BM_079...
ang esp: 7.9
ang obt: 8.3
erro: 0.4

calculando BM_266...
ang esp: 26.6
ang obt: 25.4
erro: 1.2

calculando BM_590...
ang esp: 59.0
ang obt: 58.9
erro: 0.1

calculando DI_222...
ang esp: 22.2
ang obt: 22.9
erro: 0.7

calculando DI_393...
ang esp: 39.3
ang obt: 33.6
erro: 5.7

calc